# Headers 

In [22]:
from theano.sandbox import cuda
cuda.use('gpu0')
import os, cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras import backend as K
K.set_image_dim_ordering('th')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.optimizers import SGD,RMSprop,adam

from sklearn.metrics import classification_report,confusion_matrix
import itertools

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



## Setting Data Path Directory

* some configurations for convolutional neuralnet

In [8]:
%cd ~/work/deeplearning/
batch_size=16

path="/home/asaeed9/work/data/2cat/sample/"
results_path = "/home/asaeed9/work/data/2cat/sample/results"

train_path = path + "/train"
valid_path = path + "/valid"
test_path = path + "/test"

img_rows=128
img_cols=128
num_channel=1
num_epoch=20
num_classes = 2

names = ['cats','dogs']

/home/asaeed9/work/deeplearning


* Following function takes the input as path to the data, and returns grey-scaled images and labels for those images.

In [13]:
def get_greyed_images(data_path):
    data_dir_list = os.listdir(data_path)
    img_data_list=[]
    labels=[]

    for dataset in data_dir_list:
        img_list=os.listdir(data_path+'/'+ dataset)
        print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
        for img in img_list:

            if 'cat' in img:
                labels.append(0)
            else:
                labels.append(1)


            input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
            input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
            input_img_resize=cv2.resize(input_img,(128,128))
            img_data_list.append(input_img_resize)

    img_data = np.array(img_data_list)
    img_data = img_data.astype('float32')
    img_data /= 255
    print (img_data.shape) 
    return img_data, labels

#### Get the Training Data

In [14]:
train_img_data, train_labels = get_greyed_images(train_path)
#expand a dimension to add 1 to the shape of train_img_data
train_img_data= np.expand_dims(train_img_data, axis=1) 
train_nsamples = train_img_data.shape[0]
train_input_shape=train_img_data[0].shape

train_Y = np_utils.to_categorical(train_labels, num_classes)

Loaded the images of dataset-dogs

Loaded the images of dataset-cats

(1500, 128, 128)


#### Get the Validation Set

In [15]:
valid_img_data, valid_labels = get_greyed_images(valid_path)
#expand a dimension to add 1 to the shape of train_img_data
valid_img_data= np.expand_dims(valid_img_data, axis=1) 
valid_nsamples = valid_img_data.shape[0]
valid_input_shape=valid_img_data[0].shape

valid_Y = np_utils.to_categorical(valid_labels, num_classes)

Loaded the images of dataset-dogs

Loaded the images of dataset-cats

(500, 128, 128)


#### Get the Test Data

In [16]:
test_img_data, test_labels = get_greyed_images(test_path)
test_img_data = np.expand_dims(test_img_data, axis=1)
test_nsamples = test_img_data.shape[0]
test_input_shape=test_img_data[0].shape

test_Y = np_utils.to_categorical(test_labels, num_classes)

Loaded the images of dataset-dogs

Loaded the images of dataset-cats

(3000, 128, 128)


## Build Model on Training Set

* Following function defines a model

In [20]:
def get_cnn_model(input_shape):
    model = Sequential()

    model.add(Convolution2D(32, 3,3,border_mode='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    #model.add(Convolution2D(64, 3, 3))
    #model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    #model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])  
    
    return model

* Following code calls the above model

In [21]:
model = get_cnn_model(train_input_shape)
model.summary()
hist = model.fit(train_img_data, train_Y, batch_size=16, nb_epoch=num_epoch, verbose=1, validation_data=(valid_img_data, valid_Y))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_4 (Convolution2D)  (None, 32, 128, 128)  320         convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_6 (Activation)        (None, 32, 128, 128)  0           convolution2d_4[0][0]            
____________________________________________________________________________________________________
convolution2d_5 (Convolution2D)  (None, 32, 126, 126)  9248        activation_6[0][0]               
____________________________________________________________________________________________________
activation_7 (Activation)        (None, 32, 126, 126)  0           convolution2d_5[0][0]            
___________________________________________________________________________________________

### Make Predictions on the Learned Model

In [23]:
Y_pred = model.predict(test_img_data)
print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
#y_pred = model.predict_classes(X_test)
#print(y_pred)
target_names = ['class 0(cats)', 'class 1(Dogs)']

print(classification_report(np.argmax(test_Y,axis=1), y_pred,target_names=target_names))

print(confusion_matrix(np.argmax(test_Y,axis=1), y_pred))

[[ 0.00203098  0.99796897]
 [ 0.26495466  0.73504537]
 [ 0.00867537  0.99132466]
 ..., 
 [ 0.30260459  0.69739538]
 [ 0.10978741  0.89021254]
 [ 0.9154247   0.08457533]]
[1 1 1 ..., 1 1 0]
               precision    recall  f1-score   support

class 0(cats)       0.70      0.68      0.69      1497
class 1(Dogs)       0.69      0.71      0.70      1503

  avg / total       0.70      0.70      0.70      3000

[[1022  475]
 [ 433 1070]]
